In [ ]:
import re
from pathlib import Path

def recover_original_filename(path: str) -> str:
    """
    Given a YOLO-formatted image filename with suffix '_<EXT>.rf.<HASH>.jpg',
    return the original filename: <base>.<EXT>
    """
    name = Path(path).name  # strip directory

    # Match pattern: anything + _EXT.rf.HASH.jpg
    pattern = r"^(.*)_([A-Za-z0-9]+)\.rf\.[a-f0-9]{32}\.jpg$"
    match = re.match(pattern, name)
    if not match:
        raise ValueError(f"Filename format not recognized: {name}")

    base_name, ext = match.groups()
    return f"{base_name}.{ext.upper()}"

In [ ]:
from pathlib import Path
from typing import List, Union

def write_paths_to_txt(paths: List[Union[str, Path]], output_file: Union[str, Path]):
    """
    Write a list of paths to a .txt file, one path per line.

    Args:
        paths (List[Union[str, Path]]): List of file paths.
        output_file (Union[str, Path]): Output .txt file path.
    """
    output_file = Path(output_file)
    with output_file.open('w', encoding='utf-8') as f:
        for path in paths:
            f.write(str(path) + '\n')

In [ ]:
from pathlib import Path

def modify_filename_prefix(path: str, prefix: str, add: bool = True) -> str:
    """
    Add or remove a prefix from the filename (before extension).

    Args:
        path (str): Original file path.
        prefix (str): Prefix to add or remove (no slash).
        add (bool): If True, add prefix; if False, remove it if present.

    Returns:
        str: Modified filename (not full path).
    """
    p = Path(path)
    stem = p.stem
    ext = p.suffix

    if add:
        if not stem.startswith(prefix):
            stem = prefix + stem
    else:
        if stem.startswith(prefix):
            stem = stem[len(prefix):]

    return stem + ext

In [ ]:
from pathlib import Path

def replace_filename_prefix(path: str, old_prefix: str, new_prefix: str) -> str:
    """
    Replace a specific prefix in the filename (before the extension) with another.

    Args:
        path (str): Original file path or filename.
        old_prefix (str): The prefix to be replaced.
        new_prefix (str): The prefix to insert.

    Returns:
        str: Modified filename (not full path).
    """
    p = Path(path)
    stem = p.stem
    ext = p.suffix

    if stem.startswith(old_prefix):
        stem = new_prefix + stem[len(old_prefix):]
    # else: leave unchanged

    return stem + ext

In [ ]:
#!/usr/bin/env python3

import os
import sys
import shutil

def extract_two_folder_prefix(dirpath, root_folder):
    """
    Extract the last two subfolders from dirpath *relative to* root_folder.
    Adjust as needed if your structure is different.
    """
    relative_path = os.path.relpath(dirpath, root_folder)  # the path under root_folder
    parts = relative_path.strip(os.sep).split(os.sep)
    if len(parts) >= 2:
        return f"{parts[-2]}_{parts[-1]}"
    elif len(parts) == 1:
        return parts[0]
    else:
        return "unknown"

def strip_first_two_prefixes(old_basename):
    """
    Remove the first two underscore-delimited tokens from the start of the filename base.
    e.g. "eb7a_Dibo_IMAGE2458" => "IMAGE2458"
    """
    parts = old_basename.split("_", 2)
    if len(parts) > 2:
        return parts[2]
    else:
        return old_basename

def build_new_name(dirpath, filename, root_folder):
    base, ext = os.path.splitext(filename)  # e.g. base="eb7a_Dibo_IMAGE2458", ext=".jpg"
    prefix = extract_two_folder_prefix(dirpath, root_folder)  # e.g. "Dibo_dgor1"
    remainder = strip_first_two_prefixes(base)                # e.g. "IMAGE2458"
    return f"{prefix}_{remainder}{ext}"                      # e.g. "Dibo_dgor1_IMAGE2458.jpg"

def main():
    if len(sys.argv) != 4:
        print(f"Usage: {sys.argv[0]} <root_folder> <jpg_output_folder> <txt_output_folder>")
        sys.exit(1)

    root_folder = sys.argv[1]
    jpg_out = sys.argv[2]
    txt_out = sys.argv[3]

    # Ensure output folders exist
    os.makedirs(jpg_out, exist_ok=True)
    os.makedirs(txt_out, exist_ok=True)

    # Walk through the entire directory tree under root_folder
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for filename in filenames:
            # Only handle .jpg/.jpeg/.txt
            base_lower = filename.lower()
            if not (base_lower.endswith(".jpg") or base_lower.endswith(".jpeg") or base_lower.endswith(".txt")):
                continue

            full_path = os.path.join(dirpath, filename)
            new_name = build_new_name(dirpath, filename, root_folder)

            # Decide output folder
            _, ext = os.path.splitext(base_lower)
            if ext in [".jpg", ".jpeg"]:
                out_folder = jpg_out
            else:
                out_folder = txt_out

            dest_path = os.path.join(out_folder, new_name)
            shutil.move(full_path, dest_path)
            print(f"Moved: {full_path}  -->  {dest_path}")

if __name__ == "__main__":
    main()

In [ ]:
folders = [
    "/storage/brno2/home/chlupp/sloni/datasets_src/elephants_640_man_rf_train",
    "/storage/brno2/home/chlupp/sloni/datasets_src/elephants_640_man_rf_val"
           ]

# Get all image files in the folders
files = []
for folder in folders:
    files.extend([p for p in Path(folder).iterdir() if p.is_file() and p.suffix.lower() == '.jpg'])

# Get original names of the files
names = []
for path in files:
    try:
        original_name = recover_original_filename(path)
        names.append(original_name)
        print(f"Original name: {original_name}")
    except ValueError as e:
        print(e)

# Now modify the names by adding the prefix as the original data now have - necessary to remove ambiguity
new_names = []
for name in names:
    new_name = modify_filename_prefix(name, prefix=, add=True)